In [23]:
!pip install bs4
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Fetched 252 kB in 2s (154 kB/s)
Reading package lists... Done
Reading package lists... Done
Building d

# **Importing library dependencies**

At First the dependencies are imported  - Beautiful Soup  library for parsing the HTML page and selenium and chromium driver for loading the webpage.

In [0]:
from bs4 import BeautifulSoup as soup
import codecs


In [0]:
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# **Loading pages and parsing HTML**

First the base url page is loaded and then the parsed HTML page is stored. This HTML page is used to extract the celebrity details like celebrity name, image link etc.

In [0]:
BASE_URL = "http://m.imdb.com/feature/bornondate" # As given in the problem statement
    
	# The page content is dynamic, so, a webdriver is required to emulate a browser
	# environment and run the javascript functions.

driver = webdriver.Chrome('chromedriver',options=options)
driver.get(BASE_URL)

page_html = driver.page_source
page_soup=soup(page_html,"html5lib")
boccat = page_soup.findAll("div", {"class":"lister-item mode-detail"})
driver.close()

# **Extracting the details individually and storing the details**

The HTML file is parsed and then the individual fields are extracted from the parsed HTML document using the HTML tags. In cases if either image, profession or any field is not available then they would display "Not Available" or "No Image Present."

This data is then written to a file so that it can be viewed efficiently.

After each iteration of the inner loop the next page is loaded by accessing the attribute tag avaiable for the next page and the loop terminates only if the attribute tag for the next page is unavailable. 

In [0]:
#print("Sl No    Celebrity Name   Image Link    Profession    Best Work   Early Life")
counter = 1

outputFile = codecs.open("/content/drive/My Drive/Celebrity_Details.txt", 'w', "utf-8")
while(1):
  for i in range(len(boccat)):

    celebrity_name = boccat[i].find("h3",{"class": "lister-item-header"}).a.text.strip()
    a = boccat[i].findAll("p",{"class":"text-muted text-small"})
    if(len(a)>0):
      a=a[0].text.strip().split("|")
      
      if(len(a)==2):
        profession = boccat[i].find("p",{"class":"text-muted text-small"}).text.strip().split("|")[0]
        best_work= boccat[i].find("p",{"class":"text-muted text-small"}).text.strip().split("|")[1]
      else:
        best_work=boccat[i].find("p",{"class":"text-muted text-small"}).text.strip().split("|")[0]
        profession="Not Avialable"
    else:
      profession="Not Avialable"
      best_work="Not Avialable"
    best_work = best_work.strip()
    k=str(i+1)
    image_link=boccat[i].findAll("img",{"alt":celebrity_name})
    if(len(image_link)>0):
      image=image_link[0]
      image_1= image["src"]
    else:
      image_1="No Image present"
    early_life=boccat[i].findAll("p")
    if(len(early_life)>1):

      early_life=early_life[1].text.strip()
    elif(len(early_life)>0 and len(profession)==0 and len(best_work)==0):
      early_life=early_life[0].text.strip()
    else:
      early_life="Not Avialable"
#    print(str(counter)+"   "+celebrity_name+ "    " +image_1+"   "+profession+"    "+best_work+"   "+early_life)

    outputFile.write("Serial Number : " + str(counter) + "\n")
    outputFile.write("Name of the celebrity: " + celebrity_name + "\n")
    outputFile.write("Celebrity Image: " + image_1.strip() + "\n")
    outputFile.write("Profession: " + profession + "\n")
    outputFile.write("Best Work : " + best_work + "\n")
    outputFile.write("Early Life :" + early_life + "\n")
  
    outputFile.write("\n\n")
    
    counter=counter+1

  next_page=page_soup.findAll("a",{"class":"lister-page-next next-page"})
  if(len(next_page)>0):
    next_page=next_page[0]
    next_URL=next_page["href"]
    driver = webdriver.Chrome('chromedriver',options=options)
    driver.get("https://www.imdb.com"+next_URL)


    page_html = driver.page_source
    driver.close()
    page_soup=soup(page_html,"html5lib")
    boccat = page_soup.findAll("div", {"class":"lister-item mode-detail"})
  else:
    break
outputFile.close()

